# Experimenting Notebook

This notebook aims develop and test the code that will be used on the app.

In [27]:
import nltk
import re
import string
from nltk.tokenize import sent_tokenize as nlkt_sent_tokenize
from nltk.tokenize import word_tokenize as nlkt_word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from nltk.corpus import stopwords
import numpy as np
from scipy.spatial.distance import cosine
from newspaper import Article

In [9]:
url_test = 'https://economia.uol.com.br/mais/ultimas-noticias/2024/02/02/dividendos-no-bolso-todo-mes-veja-16-acoes-que-fazem-pagamentos-regulares.htm'

# Testing the extraction of the text of an article from its URL 
def get_text_from_article(article_url):
    article = Article(article_url)
    article.download()
    article.parse()
    return article.text

print(get_text_from_article(url_test))

É uma opção para quem busca proventos no longo prazo, diz Milton Rabelo, analista do setor financeiro da VG Research. Isso porque o banco enfrentou momentos difíceis recentemente por conta da inadimplência. "Ficou muito claro que houve grandes desajustes na concessão de crédito e nas operações de tesouraria e, desde então, a gestão tem buscado corrigir esses equívocos. Porém essa mudança de rota leva um tempo", avalia Rabelo.

Com uma recuperação mais lenta, as ações do banco estariam com desconto, abrindo uma oportunidade. Rabelo projeta um dividend yield, ou seja, retorno com dividendos sobre o preço da ação, de 7% para as ações BBDC4 em 2024.

Outro banco que faz remunerações mensais é o Itaú. Rabelo destaca que o Itaú é um banco com sólidos fundamentos, grande valor de mercado, líder e com crescimento nos lucros e receitas.

A desvantagem é que justamente por esse motivo as ações podem estar caras. Em consequência o dividend yield (retorno em dividendos, que é a relação entre o div

In [10]:
nltk.download("punkt")
nlkt_sent_tokenize(get_text_from_article(url_test))

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['É uma opção para quem busca proventos no longo prazo, diz Milton Rabelo, analista do setor financeiro da VG Research.',
 'Isso porque o banco enfrentou momentos difíceis recentemente por conta da inadimplência.',
 '"Ficou muito claro que houve grandes desajustes na concessão de crédito e nas operações de tesouraria e, desde então, a gestão tem buscado corrigir esses equívocos.',
 'Porém essa mudança de rota leva um tempo", avalia Rabelo.',
 'Com uma recuperação mais lenta, as ações do banco estariam com desconto, abrindo uma oportunidade.',
 'Rabelo projeta um dividend yield, ou seja, retorno com dividendos sobre o preço da ação, de 7% para as ações BBDC4 em 2024.',
 'Outro banco que faz remunerações mensais é o Itaú.',
 'Rabelo destaca que o Itaú é um banco com sólidos fundamentos, grande valor de mercado, líder e com crescimento nos lucros e receitas.',
 'A desvantagem é que justamente por esse motivo as ações podem estar caras.',
 'Em consequência o dividend yield (retorno em divi

In [13]:
# Testing the normalizing step
def sent_tokenize(text):
    nltk.download("punkt")
    return nlkt_sent_tokenize(text)


def normalizing_sentence(text):
    nltk.download("stopwords")
    stop_words = stopwords.words('portuguese')
    sentences = sent_tokenize(text)
    sentences_cleaned = []
    for sentence in sentences:
        words = nlkt_word_tokenize(sentence)
        words = [w for w in words if w not in string.punctuation]
        words = [w for w in words if not w.lower() in stop_words]
        words = [w.lower() for w in words]
        sentences_cleaned.append(" ".join(words))
    return sentences_cleaned

In [31]:
vectorizer = TfidfVectorizer()
sent_word_matrix = vectorizer.fit_transform(normalizing_sentence(get_text_from_article(url_test)))
sent_word_matrix.toarray().shape

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(19, 166)

In [44]:
# Testing the TF_IDF transformation
def get_tfidf_matrix(sentences):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    return tfidf_matrix.toarray(), vectorizer.get_feature_names_out()
    
def get_centroid_words(tfidf_matrix, feature_names):
    centroid_vector = tfidf_matrix.sum(0)
    centroid_vector = np.divide(centroid_vector, centroid_vector.max())
    relevant_vector_indices = np.where(centroid_vector > 0.3)[0]
    word_list = list(np.array(feature_names)[relevant_vector_indices])
    return word_list

In [46]:
# Calculates cosine similarity
def similarity(v1, v2):
    score = 0.0
    if np.count_nonzero(v1) != 0 and np.count_nonzero(v2) != 0:
        score = ((1 - cosine(v1, v2)) + 1) / 2
    return score

In [45]:
obj1, obj2 = get_tfidf_matrix(normalizing_sentence(get_text_from_article(url_test)))
get_centroid_words(obj1, obj2)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['2024',
 'aeroporto',
 'além',
 'assim',
 'ações',
 'banco',
 'banestes',
 'catarina',
 'crescimento',
 'dividend',
 'dividendos',
 'dy',
 'faz',
 'investidores',
 'itaú',
 'lucro',
 'menos',
 'mensais',
 'opção',
 'outro',
 'paga',
 'pode',
 'preço',
 'projeta',
 'proventos',
 'rabelo',
 'remunerações',
 'retorno',
 'setor',
 'sobre',
 'yield']

In [11]:
# Populate word vector with all embeddings.
# This word vector is a look up table that is used
# for getting the centroid and sentences embedding representation.
def word_vectors_cache(sentences, embedding_model):
    word_vectors = dict()
    for sent in sentences:
        words = nlkt_word_tokenize(sent)
        for w in words:
            word_vectors.update({w: embedding_model.wv[w]})
    return word_vectors


# Sentence embedding representation with sum of word vectors
def build_embedding_representation(words, word_vectors, embedding_model):
    embedding_representation = np.zeros(embedding_model.vector_size, dtype="float32")
    word_vectors_keys = set(word_vectors.keys())
    count = 0
    for w in words:
        if w in word_vectors_keys:
            embedding_representation = embedding_representation + word_vectors[w]
            count += 1
    if count != 0:
        embedding_representation = np.divide(embedding_representation, count)
    return embedding_representation


def summarize(text, emdedding_model):
    raw_sentences = sent_tokenize(text)
    clean_sentences = cleanup_sentences(text)
    for i, s in enumerate(raw_sentences):
        print(i, s)
    for i, s in enumerate(clean_sentences):
        print(i, s)
    centroid_words = get_tf_idf(clean_sentences)
    print(len(centroid_words), centroid_words)
    word_vectors = word_vectors_cache(clean_sentences, emdedding_model)
    # Centroid embedding representation
    centroid_vector = build_embedding_representation(centroid_words, word_vectors, emdedding_model)
    sentences_scores = []
    for i in range(len(clean_sentences)):
        scores = []
        words = clean_sentences[i].split()

        # Sentence embedding representation
        sentence_vector = build_embedding_representation(words, word_vectors, emdedding_model)

        # Cosine similarity between sentence embedding and centroid embedding
        score = similarity(sentence_vector, centroid_vector)
        sentences_scores.append((i, raw_sentences[i], score, sentence_vector))
    sentence_scores_sort = sorted(sentences_scores, key=lambda el: el[2], reverse=True)
    for s in sentence_scores_sort:
        print(s[0], s[1], s[2])
    count = 0
    sentences_summary = []
    # Handle redundancy
    for s in sentence_scores_sort:
        if count > 100:
            break
        include_flag = True
        for ps in sentences_summary:
            sim = similarity(s[3], ps[3])
            if sim > 0.95:
                include_flag = False
        if include_flag:
            sentences_summary.append(s)
            count += len(s[1].split())

        sentences_summary = sorted(sentences_summary, key=lambda el: el[0], reverse=False)

    summary = "\n".join([s[1] for s in sentences_summary])
    return summary